In [1]:
import re
import json
import pandas as pd
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup

from blog_fake_review.constant import TagInfo
from blog_fake_review.parser import extract_article_body, refine_paragraph_tag, extract_paragraph

In [35]:
origins = glob("./data/origin/*.json")
origins

['./data/origin/홍대 회식 맛집.json',
 './data/origin/홍대 데이트 맛집.json',
 './data/origin/성수 데이트 맛집.json',
 './data/origin/압구정 회식 맛집.json',
 './data/origin/강남역 데이트 맛집.json',
 './data/origin/영등포 회식 맛집.json',
 './data/origin/성수 회식 맛집.json',
 './data/origin/압구정 데이트 맛집.json',
 './data/origin/강남역 회식 맛집.json',
 './data/origin/범계 데이트 맛집.json',
 './data/origin/영등포 데이트 맛집.json']

In [36]:
parsed_list = glob("./data/semi-structured/*.json")
parsed_list = [p.split('/')[-1] for p in parsed_list]
parsed_list

['홍대 회식 맛집.json',
 '홍대 데이트 맛집.json',
 '성수 데이트 맛집.json',
 '압구정 회식 맛집.json',
 '강남역 데이트 맛집.json',
 '영등포 회식 맛집.json',
 '성수 회식 맛집.json',
 '압구정 데이트 맛집.json',
 '강남역 회식 맛집.json',
 '영등포 데이트 맛집.json']

In [37]:
comment_code = re.compile(r'<!--.*?-->')

In [38]:
for path in origins:
    file_name = path.split('/')[-1]
    
    if file_name in parsed_list: continue
        
    with open(path, "r", encoding='utf-8') as f:
        blog = json.load(f)

    result = []
    for row in tqdm(blog, desc=file_name.split('.')[0]):
        data = {
            "blog_name": row["blog_name"],
            "date": row["date"],
            "url": row["url"],
            "title": row["title"]
        }
        
        soup = BeautifulSoup(comment_code.sub('', row['origin_source']), 'html.parser')
        version, body_tag = extract_article_body(soup)
        
        if not body_tag: continue
            
        tags = extract_paragraph(body_tag, version)
        
        data["contents"] = sum([refine_paragraph_tag(tag) for tag in tags], [])
        
        result.append(data)
        
    with open(f'./data/semi-structured/{file_name}', 'w', encoding='utf-8') as f:
        json.dump(result, f)

범계 데이트 맛집: 100%|██████████| 985/985 [01:02<00:00, 15.88it/s]


In [39]:
with open(f'./data/semi-structured/{parsed_list[0]}', 'r', encoding='utf-8') as f:
    blog = json.load(f)

In [42]:
blog[130]

{'blog_name': '굴링굴링 굴러가는 하루',
 'date': '20221113',
 'url': 'https://blog.naver.com/li_muming/222927372676',
 'title': "[<b>홍대</b>입구역 <b>맛집</b>] <b>홍대</b> 소곱창/막창 <b>회식</b>하기 좋은 <b>맛집</b> '황곱'",
 'contents': [{'info': 'text',
   'content': '이제 슬슬 날씨도 추워지고 녹진한 곱창이나 막창같은,\n위장에 기름칠만한 메뉴들이 생각나는 계절이 왔어요\n저는 오늘 홍대입구역에 다녀왔는데요!\n여긴 정말 지나다닐때마다 여긴 뭐길래 이렇게 사람이 많지.. 싶었는데\n알고보니 곱창구이 집이더라고요.\n\n저는 홍대 근처에 자주 오지만 항상 연남동에서나 식사를 했거든요\n웨이팅하는 거 너무 힘들잖아요 ... ㅋㅋㅋㅋㅋㅋ\n그런데 왜 웨이팅하는지 알겠더라고요?!\n\n진짜 어떻게 하신거죠?!ㅋㅋㅋㅋㅋㅋ\n제가 곱창을 정말 좋아해서 맛집이란 곳은 다 돌아다녔거든요\n황곱은 방송에 아주 많이 나온 집도 아닌데, 홍대 곱창집 중 가장 맛있는 것 같아요\n홍대 곱창집이 아니라 그냥 서울 곱창집 중에서?\n소곱창이 이렇게 맛있을 수 있다는 게 신기해서 홍대 소곱창 맛집 황곱 소개해드리려고 해요!'},
  {'info': 'text',
   'content': '영업시간\n일~목 17:00 - 23:00\n금~토 15:00 - 23:00\n\n단체석, 와이파이 O\n화장실 O'},
  {'info': 'img',
   'content': 'https://postfiles.pstatic.net/MjAyMjExMTNfMTQ4/MDAxNjY4MzA2ODA1NjM3.1scx5SZE39-AcQYmLR-RqrQl3VE_y9294IvMz5rcCC0g.OdBZlMvNwtDpqL_OJNBBtaBx5SLRMoaIha-mOSr5TFgg.JPEG.li_muming/20221112%EF%BC%BF165953.jpg?type=w